In [19]:
import os
import requests
from typing import Annotated
from dotenv import load_dotenv
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_core.tools import InjectedToolArg
from langchain_core.messages import HumanMessage


In [20]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [21]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/d5ce7027b03ad3604a72e903/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [22]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'BDT'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1760227202,
 'time_last_update_utc': 'Sun, 12 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1760313602,
 'time_next_update_utc': 'Mon, 13 Oct 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 121.8002}

In [23]:
convert.invoke({'base_currency_value':10, 'conversion_rate':121.8002})

1218.002

In [24]:
LLM = ChatGroq(groq_api_key=os.environ.get("GROQ_API_KEY"), model="openai/gpt-oss-120b", temperature=1.2)
LLM_with_tool = LLM.bind_tools([get_conversion_factor, convert])

In [26]:
messages = [HumanMessage('What is the conversion factor between BDT and USD, and based on that can you convert 10 BDT to USD?')]

In [27]:
ai_message = LLM_with_tool.invoke(messages)

In [28]:
messages.append(ai_message)

In [29]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'BDT', 'target_currency': 'USD'},
  'id': 'fc_46e89100-4f1d-471e-ada5-fe44d4767e0c',
  'type': 'tool_call'}]

In [30]:
import json

for tool_call in ai_message.tool_calls:
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1)
  if tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [31]:
messages

[HumanMessage(content='What is the conversion factor between BDT and USD, and based on that can you convert 10 BDT to USD?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to fetch conversion factor between BDT and USD. Use function get_conversion_factor with base_currency "BDT", target_currency "USD". Then calculate conversion of 10 BDT to USD using convert with base_currency_value 10, but we need to provide the factor? The convert function presumably uses some stored factor? It takes base_currency_value but no factor; maybe it uses a previously fetched factor stored context? Unclear. Likely we need to call get_conversion_factor first, then use convert with base_currency_value 10 and maybe need to specify factor? The description: "given a currency conversion rate this function calculates the target currency value from a given base currency value". It takes base_currency_value only, but perhaps it uses the previously

In [32]:
LLM_with_tool.invoke(messages).content

'The conversion factor from Bangladeshi Taka (BDT) to U.S. Dollars (USD) is **0.00821\u202fUSD per\u202fBDT**.\n\nTo convert 10\u202fBDT to USD:\n\n\\[\n10 \\text{ BDT} \\times 0.00821 \\frac{\\text{USD}}{\\text{BDT}} = 0.0821 \\text{ USD}\n\\]\n\n**So, 10\u202fBDT ≈\u202f$0.082\u202fUSD** (about 8.2\u202fcents).'

In [35]:
from langchain.agents import initialize_agent, AgentType

agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=LLM,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True 
)

In [36]:
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you! How can I assist you today?"
}
```

> Finished chain.
